In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/오징어게임_imdb.csv')
df1 = df1.rename(columns={'review':'reviews'})

In [3]:
df2 = pd.read_csv('/content/drive/MyDrive/오징어게임_my.csv')
df2 = df2[['Unnamed: 0', 'title', 'reviews']]

In [4]:
df3 = pd.read_csv('/content/drive/MyDrive/reviews_csv/Squid_Game_letterboxd.csv')

In [5]:
df = pd.concat([df1, df2, df3])

In [7]:
pip install langdetect

     |████████████████████████████████| 981 kB 10.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=46a062a8fdc45205af0c07de092015ec0bc3653939a531eb94b79aed1954e5dc
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [8]:
from langdetect import detect

In [9]:
from tqdm import tqdm
import pickle
import csv
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize

In [10]:
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang

In [11]:
df['Lang'] = df['reviews'].apply(det)

In [ ]:
df

In [13]:
en_reviews = (df['Lang'] == 'en')
df_en_reviews = df[en_reviews]['reviews']
len(df_en_reviews)

7146

In [23]:
detect_list = df_en_reviews.to_list()

In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [20]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

In [21]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [24]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        # pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        pos_words = []
        for word in tag_words:
            if word[1] in ['NNP', 'NN']:  #'VB', 'VBP', 'JJ'
                pos_words.append(word)
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results

In [25]:
lda_tokens = []
custom_tokenizer = CustomTokenizer(text_to_word_sequence)
for sentences in detect_list:
    lda_tokens.append(custom_tokenizer(sentences))
lda_tokens[0]

['character', 'room', 'man', 'hair', 'board', 'plane', 'beginning']

In [26]:
len(lda_tokens)

7146

In [27]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [28]:
dictionary = corpora.Dictionary(lda_tokens)

2022-04-20 11:56:34,186 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-04-20 11:56:34,415 : INFO : built Dictionary(7495 unique tokens: ['beginning', 'board', 'character', 'hair', 'man']...) from 7146 documents (total 86523 corpus positions)


In [29]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

2022-04-20 11:56:43,579 : INFO : discarding 3893 tokens: [('fluent', 1), ('flak', 1), ('steal', 1), ('apocalyptic', 1), ('blog', 1), ('denial', 1), ('enforcement', 1), ('grinding', 1), ('hault', 1), ('hopefuly', 1)]...
2022-04-20 11:56:43,582 : INFO : keeping 3602 tokens which were in no less than 2 and no more than 3573 (=50.0%) documents
2022-04-20 11:56:43,587 : INFO : resulting dictionary: Dictionary(3602 unique tokens: ['beginning', 'board', 'character', 'hair', 'man']...)


In [30]:
corpus = [dictionary.doc2bow(text) for text in lda_tokens]

In [31]:
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

2022-04-20 11:56:59,425 : INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2022-04-20 11:56:59,428 : INFO : using serial LDA version on this node
2022-04-20 11:56:59,433 : INFO : running online (multi-pass) LDA training, 4 topics, 20 passes over the supplied corpus of 7146 documents, updating model once every 2000 documents, evaluating perplexity every 2000 documents, iterating 400x with a convergence threshold of 0.001000
2022-04-20 11:57:01,969 : INFO : -8.880 per-word bound, 471.2 perplexity estimate based on a held-out corpus of 2000 documents with 31968 words
2022-04-20 11:57:01,971 : INFO : PROGRESS: pass 0, at document #2000/7146
2022-04-20 11:57:03,603 : INFO : optimized alpha [0.11245406, 0.19389662, 0.12881228, 0.17803767]
2022-04-20 11:57:03,605 : INFO : merging changes from 2000 documents into a model of 7146 documents
2022-04-20 11:57:03,610 : INFO : topic #0 (0.112): 0.047*"game" + 0.031*"series" + 0.023*"show" + 0.022*"story" + 0.018*"season" + 0.018*

In [32]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-04-20 11:58:28,352 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-04-20 11:58:28,392 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-04-20 11:58:28,406 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-04-20 11:58:28,415 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-04-20 11:58:28,448 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-04-20 11:58:28,462 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-04-20 11:58:28,494 : INFO : CorpusAccumulator accumulated stats from 7000 documents


Average topic coherence: -2.0440.
[([(0.079370975, 'show'),
   (0.033813767, 'game'),
   (0.024163026, 'episode'),
   (0.015959108, 'plot'),
   (0.01376444, 'way'),
   (0.013500706, 'time'),
   (0.012051776, 'character'),
   (0.012000385, 'story'),
   (0.011725138, 'lot'),
   (0.011602177, 'season'),
   (0.010404116, 'something'),
   (0.008803506, 'money'),
   (0.008594964, 'series'),
   (0.0082912715, 'thing'),
   (0.008117252, 'end'),
   (0.0077461763, 'life'),
   (0.0071558384, 'world'),
   (0.0069589363, 'acting'),
   (0.0069503994, 'everything'),
   (0.0069375704, 'design')],
  -1.7793576458892475),
 ([(0.05696034, 'game'),
   (0.020901224, 'episode'),
   (0.016428815, 'man'),
   (0.016163327, 'show'),
   (0.014140893, 'time'),
   (0.012505009, 'series'),
   (0.011947628, 'end'),
   (0.0115674585, 'character'),
   (0.010743644, 'jung'),
   (0.009980548, 'hun'),
   (0.009486566, 'byeok'),
   (0.007916019, 'season'),
   (0.0074657337, 'money'),
   (0.0070024957, 'light'),
   (0.0065

In [33]:
%%capture
!pip install pyldavis

In [34]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

2022-04-20 11:59:03,220 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2022-04-20 11:59:03,262 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [35]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
